In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
text_df = pd.read_csv("training_subset.csv")
text_df.head()

,submission_title,comment,comment_id,url,subreddit_id,label1,label2
0,Shashi Tharoor Set To Run For Congress Preside...,Extremely valid points but I believe he has th...,ip5g6vu,/r/india/comments/xif8wm/shashi_tharoor_set_to...,2qh1q,-1,-1
1,Yogi government puts Kolkata's image as part o...,Even if the ad was designed by the newspaper -...,hcontm8,/r/india/comments/pmn9o3/yogi_government_puts_...,2qh1q,-1,-1
2,"An attempt to address the list of ""simplified""...",Diverse population including Muslims. Welcomin...,fbhlv40,/r/india/comments/ebdeup/an_attempt_to_address...,2qh1q,-1,-2
3,You guys noticing what's happening in Sri Lanka?,>The blockade was done by your own people.\n\n...,i2yrud3,/r/india/comments/tt1ryh/you_guys_noticing_wha...,2qh1q,0,0
4,Just got abused on the train by a hyper-nation...,Ohoo bahut bura laga ye sunke ki aap Undergarm...,hooi92k,/r/india/comments/rh2kcs/just_got_abused_on_th...,2qh1q,-2,-2


In [5]:
text_df.describe()

,label1,label2
count,119.000000,119.000000
mean,-0.058824,-0.201681
std,1.043902,0.916698
min,-2.000000,-2.000000
25%,-1.000000,-1.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,2.000000,2.000000


In [6]:
text_df['avg_score'] = text_df.apply(lambda x: np.mean(x[5:]), axis=1)
text_df.head()

,submission_title,comment,comment_id,url,subreddit_id,label1,label2,avg_score
0,Shashi Tharoor Set To Run For Congress Preside...,Extremely valid points but I believe he has th...,ip5g6vu,/r/india/comments/xif8wm/shashi_tharoor_set_to...,2qh1q,-1,-1,-1.0
1,Yogi government puts Kolkata's image as part o...,Even if the ad was designed by the newspaper -...,hcontm8,/r/india/comments/pmn9o3/yogi_government_puts_...,2qh1q,-1,-1,-1.0
2,"An attempt to address the list of ""simplified""...",Diverse population including Muslims. Welcomin...,fbhlv40,/r/india/comments/ebdeup/an_attempt_to_address...,2qh1q,-1,-2,-1.5
3,You guys noticing what's happening in Sri Lanka?,>The blockade was done by your own people.\n\n...,i2yrud3,/r/india/comments/tt1ryh/you_guys_noticing_wha...,2qh1q,0,0,0.0
4,Just got abused on the train by a hyper-nation...,Ohoo bahut bura laga ye sunke ki aap Undergarm...,hooi92k,/r/india/comments/rh2kcs/just_got_abused_on_th...,2qh1q,-2,-2,-2.0


In [7]:
text_df.describe()

,label1,label2,avg_score
count,119.000000,119.000000,119.000000
mean,-0.058824,-0.201681,-0.130252
std,1.043902,0.916698,0.914715
min,-2.000000,-2.000000,-2.000000
25%,-1.000000,-1.000000,-0.500000
50%,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.500000
max,2.000000,2.000000,2.000000


## Rule-based classifier

## BERT-based classifier